In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define the ticker symbol
ticker_symbol = 'AAPL'  # Example: Apple Inc.

TEST_START_DATE = '2011-01-01'
TEST_END_DATE = '2022-10-01'




In [ ]:
permnos = {
    # "AMGN": "10321",
    "AAPL": "10001",
    # "HON": "10245", # TODO HAs NAN for at so I am leaving it out
    "MSFT": "10107",
    # "NKE": "10387",
    # "V": "10412",
    "IBM": "12490",
    # "INTC": "12585",
    # "WMT": "12576",
    "BA": "12572",
    "PG": "12590"
}


# Read the CSV file into a DataFrame
beta_data = pd.read_csv("../beta_data.csv", dtype=str)


beta_data['date'] = pd.to_datetime(beta_data['date'])
# Display the DataFrame

# Convert the dictionary to a DataFrame 
df = pd.DataFrame(list(permnos.items()), columns=['ticker', 'permno'])



merged_df_beta = pd.merge(df, beta_data, on='permno', how='inner')

merged_df_beta['tic'] = merged_df_beta['ticker']

print(merged_df_beta)

     ticker permno       month         beta_monthly       date   tic
0      MSFT  10107  1990-03-01   1.3940595837724659 1990-03-01  MSFT
1      MSFT  10107  1990-04-01    1.381873027955774 1990-04-01  MSFT
2      MSFT  10107  1990-05-01    1.431461915013959 1990-05-01  MSFT
3      MSFT  10107  1990-06-01    1.430795646350484 1990-06-01  MSFT
4      MSFT  10107  1990-07-01   1.4548354982380056 1990-07-01  MSFT
...     ...    ...         ...                  ...        ...   ...
1049    IBM  12490  2013-08-01   0.6762984733496086 2013-08-01   IBM
1050    IBM  12490  2013-09-01   0.6844924160720623 2013-09-01   IBM
1051    IBM  12490  2013-10-01   0.5463170573827856 2013-10-01   IBM
1052    IBM  12490  2013-11-01   0.4853235103051581 2013-11-01   IBM
1053    IBM  12490  2013-12-01  0.48726470248083065 2013-12-01   IBM

[1054 rows x 6 columns]


In [93]:
# Count the number of occurrences of each value in the 'category' column
value_counts = merged_df_beta.groupby('tic').size().reset_index(name='counts')

# Display the resulting DataFrame
print(value_counts)

    tic  counts
0   IBM     648
1  MSFT     406


In [94]:
grouped = merged_df_beta.groupby('tic')['date'].agg(['min', 'max']).reset_index()
print(grouped)

    tic        min        max
0   IBM 1964-01-01 2021-11-01
1  MSFT 1990-03-01 2023-12-01


# Now set up RL training

In [95]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline

# from finrl.agents.portfolio_optimization.architectures import EIIE

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

from pprint import pprint

# import sys
# sys.path.append("../FinRL-Library")

import itertools

In [96]:
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)

check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

In [97]:
TRAIN_START_DATE = '2001-01-01'
TRAIN_END_DATE = '2015-12-31'
TEST_START_DATE = '2016-01-01'
TEST_END_DATE = '2017-07-01'


from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.config_tickers import DOW_30_TICKER


# No time window needed for PPO
# TIME_WINDOW = 25
COMMISSION_FEE_PERCENT = 0.001
INITIAL_CASH = 1000000

raw_df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TEST_END_DATE,
                     ticker_list = DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (117669, 8)


In [98]:
print(raw_df)

              date        open        high         low       close     volume  \
0       2001-01-02    0.265625    0.272321    0.260045    0.224067  452312000   
1       2001-01-02   65.875000   66.500000   62.375000   44.570110    6449200   
2       2001-01-02   47.703850   47.867970   45.132656   32.608456    4394265   
3       2001-01-02   65.312500   65.312500   60.562500   40.507774    3762500   
4       2001-01-02   23.625000   23.625000   22.812500   12.591929    3285000   
...            ...         ...         ...         ...         ...        ...   
117664  2017-06-30  185.149994  186.190002  184.929993  166.607086    2392000   
117665  2017-06-30   94.379997   94.620003   93.779999   89.051460    9036600   
117666  2017-06-30   44.619999   44.750000   44.360001   30.279640   13368900   
117667  2017-06-30   79.500000   79.589996   77.139999   57.461819    7097300   
117668  2017-06-30   25.360001   25.423332   25.223333   22.200115   20889900   

         tic  day  
0      

In [99]:
# Floor the months
raw_df['date'] = pd.to_datetime(raw_df['date'])

# Create new columns 'year_month' with just the year and month 
raw_df['year_month'] = raw_df['date'].dt.to_period('M').dt.to_timestamp() 
merged_df_beta['year_month'] = merged_df_beta['date'].dt.to_period('M').dt.to_timestamp() 
# Perform an inner join on the 'year_month' 
joined_at_raw = pd.merge(raw_df, merged_df_beta, on=['tic', 'year_month'], how='inner')

joined_at_raw = joined_at_raw[['date_x', 'open', 'high', 'low', 'close', 'volume', 'tic', 'beta_monthly']]
joined_at_raw.rename(columns={'date_x': 'date'}, inplace=True)
print(joined_at_raw)

joined_at_raw['beta_monthly'] = joined_at_raw['beta_monthly'].astype(float)
print(joined_at_raw.dtypes)

           date       open       high        low      close     volume   tic  \
0    2001-01-02  80.783936  83.652008  80.425430  43.262703    8375531   IBM   
1    2001-01-02  22.062500  22.500000  21.437500  13.372381   82413200  MSFT   
2    2001-01-03  80.066925  90.822182  80.066925  48.268028   13357315   IBM   
3    2001-01-03  21.593750  24.437500  21.562500  14.778985  135962200  MSFT   
4    2001-01-04  90.583176  95.363289  88.611374  47.534782   16129843   IBM   
...         ...        ...        ...        ...        ...        ...   ...   
7414 2017-06-26  71.400002  71.709999  70.440002  64.805046   19607000  MSFT   
7415 2017-06-27  70.110001  70.180000  69.180000  63.592216   25215100  MSFT   
7416 2017-06-28  69.209999  69.839996  68.790001  64.134323   25806200  MSFT   
7417 2017-06-29  69.379997  69.489998  68.089996  62.930656   28918700  MSFT   
7418 2017-06-30  68.779999  69.379997  68.739998  63.334953   24161100  MSFT   

            beta_monthly  
0     1.1070

In [100]:
nan_rows_count = joined_at_raw.isna().any(axis=1).sum()
print(f'Number of rows with NaN values: {nan_rows_count}')
assert nan_rows_count == 0

Number of rows with NaN values: 0


In [101]:
import pandas as pd

# Count the number of occurrences of each value in the 'category' column
value_counts = joined_at_raw.groupby('tic').size().reset_index(name='counts')

# Display the resulting DataFrame
print(value_counts)


    tic  counts
0   IBM    3269
1  MSFT    4150


In [102]:
import pandas as pd


import pandas as pd


# Specify the column of interest
column = 'beta_monthly'

# Calculate the range of column values
column_min = joined_at_raw[column].min()
column_max = joined_at_raw[column].max()

# Display the range of column values
print(f"Range of '{column}' values: min={column_min}, max={column_max}")

# Identify outliers using the IQR method
Q1 = joined_at_raw[column].quantile(0.25)
Q3 = joined_at_raw[column].quantile(0.75)
IQR = Q3 - Q1

# Define outliers as values below Q1 - 1.5*IQR or above Q3 + 1.5*IQR
outliers = joined_at_raw[(joined_at_raw[column] < (Q1 - 1.5 * IQR)) | (joined_at_raw[column] > (Q3 + 1.5 * IQR))]

# Display the outliers
print(f"\nOutliers in '{column}' column:")
print(outliers)



Range of 'beta_monthly' values: min=0.4853235103051581, max=1.72549210885349

Outliers in 'beta_monthly' column:
Empty DataFrame
Columns: [date, open, high, low, close, volume, tic, beta_monthly]
Index: []
